In [ ]:
#Basic Liberaries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)
#Visualization
import matplotlib.pyplot as plt
import seaborn as sns
#stop warning
import warnings
warnings.filterwarnings('ignore')
#File Paths
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<h2 style="background-color:powderblue; color:red; font-family:verdana;text-align:center;border: 20px solid black;margin: 50px;font-weight: bold;">
   <br> Data Manupilation Pipeline<br> 

In [ ]:
#read the CSV
df = pd.read_csv('/kaggle/input/customer-personality-analysis/marketing_campaign.csv' ,sep = '\t')
#Drop un-needed columns
df.drop(['ID' , 'Z_CostContact' , 'Z_Revenue' , 'Recency' ,'Marital_Status' ,'Education' ] 
        , axis = 1 , inplace = True)
#convert the customer enrolled date to Datetime type
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])
# Calculate the number of days from customer enrollement to Jan 2015
df['uptoddate'] = '01-01-2015'
df['uptoddate'] = pd.to_datetime(df['uptoddate'])
df['number_days_from_Enrollment'] = (df['uptoddate'] - df['Dt_Customer']).dt.days
#Drop the customer enrollement date and the uptodate column
df.drop([ 'uptoddate' , 'Dt_Customer' ]  , axis = 1 , inplace = True)
#binaries the kids and teen columns
df.Kidhome = df.Kidhome.apply(lambda x: 1 if x == 2 else x)
df.Teenhome = df.Teenhome.apply(lambda x: 1 if x == 2 else x)
#Normalized the right skewed columns
df['log_MntWines'] = np.log1p(df.MntWines)
df['log_MntFruits']= np.log1p(df.MntFruits)
df['log_MntMeatProducts']= np.log1p(df.MntMeatProducts)
df['log_MntFishProducts']= np.log1p(df.MntFishProducts)
df['log_MntSweetProducts']= np.log1p(df.MntSweetProducts)
df['log_MntGoldProds']= np.log1p(df.MntGoldProds)
df['log_NumDealsPurchases'] = np.log1p(df.NumDealsPurchases)
df['log_NumWebPurchases'] = np.log1p(df.NumWebPurchases)
df['log_NumCatalogPurchases'] = np.log1p(df.NumCatalogPurchases)
df['log_NumStorePurchases'] = np.log1p(df.NumStorePurchases)
df['log_NumWebVisitsMonth'] = np.log1p(df.NumWebVisitsMonth)
df.drop(['MntWines' , 'MntFruits' , 'MntMeatProducts' , 'MntFishProducts' , 
         'MntSweetProducts' , 'MntGoldProds' , 'NumDealsPurchases' , 
        'NumWebPurchases' , 'NumCatalogPurchases' , 'NumStorePurchases' , 
        'NumWebVisitsMonth' ] , axis = 1 , inplace = True)
#remove the outliers in the Year Birth column and replace it with the mean
df['Year_Birth'] = df['Year_Birth'].apply(lambda x : 1973 if x< 1940  else x)
#create age band for the customer
df['older_than_50_years_old'] = df['Year_Birth'].apply(lambda x :1 if x<1965 else 0)
df['from_30_to_50_years_old'] = df['Year_Birth'].apply(lambda x :1 if 1965<x<1985 else 0)
df['younge_than_30'] = df['Year_Birth'].apply(lambda x :1 if 1985<x else 0)
df.drop([ 'Year_Birth'] , axis = 1 , inplace = True)
#create Log column for incume and fill the missing with the mean 
df['log_Income'] = np.log1p(df.Income)
df['log_Income'] = df['log_Income'].fillna(df['log_Income'].mean())
#Drop income column
df.drop('Income' , axis  = 1 , inplace  = True)
#create new column for the total expenses per customer
df['Total_expenses'] = df.log_MntWines+df.log_MntFruits+df.log_MntMeatProducts+df.log_MntFishProducts+df.log_MntSweetProducts+df.log_MntGoldProds
#create a new column for the expenses rat per day
df['expenses_per_day'] = df.Total_expenses / df.number_days_from_Enrollment
#Create bands for the number of days form enrollement
df['new_enrollement_less_year'] = df.number_days_from_Enrollment.apply(lambda x: 1 if x <300 else 0)
df['Range_one_year_enrollment'] = df.number_days_from_Enrollment.apply(lambda x: 1 if 366<x <510 else 0)
df['Two_years_enrollement'] = df.number_days_from_Enrollment.apply(lambda x: 1 if 510<x <730 else 0)
df['Long_term_enrollement'] = df.number_days_from_Enrollment.apply(lambda x: 1 if 730<x  else 0)
df.drop('number_days_from_Enrollment' , axis  = 1 , inplace  = True)
#create bands for expenses per day rate
df['low_expenses/day_rate'] = df.expenses_per_day.apply(lambda x : 1 if x<0.023320 else 0)
df['good_expenses/day_rate'] = df.expenses_per_day.apply(lambda x : 1 if 0.023320<x<0.051404 else 0)
df['Excellent_expenses/day_rate'] = df.expenses_per_day.apply(lambda x : 1 if 0.051404<x else 0)
df.drop('expenses_per_day' , axis  = 1 , inplace  = True)
#for the education column add the 2nd cycle to the Master
"""

df['Education'] = df.Education.apply(lambda x: 'Master' if x == '2n Cycle' else x )
#adjust the Maritalstatus column
#https://www.kaggle.com/miguelfzzz/store-customers-clustering-analysis
df['Marital_Status'] = df['Marital_Status'].str.replace('Married', 'In relationship')
df['Marital_Status'] = df['Marital_Status'].str.replace('Together', 'In relationship')
df['Marital_Status'] = df['Marital_Status'].str.replace('Divorced', 'Single')
df['Marital_Status'] = df['Marital_Status'].str.replace('Widow', 'Single')
df['Marital_Status'] = df['Marital_Status'].str.replace('Absurd', 'Single')
df['Marital_Status'] = df['Marital_Status'].str.replace('Alone', 'Single')
df['Marital_Status'] = df['Marital_Status'].str.replace('YOLO', 'Single')
#Get dummies form the Education and marital status column
df = pd.get_dummies(data=df, columns=['Education'  , 'Marital_Status'])
"""
df.head()

<h2 style="background-color:powderblue; color:red; font-family:verdana;text-align:center;border: 20px solid black;margin: 50px;font-weight: bold;">
   <br> Quick visualization for the distribution of the new generated dataframe<br> 

In [ ]:
fig , ax = plt.subplots(5 , 7 , figsize = (20,20))
for axes  , column in zip(ax.flatten() ,df.columns ):
    plt.style.use('seaborn-dark-palette')
    sns.histplot(data  = df , x =column  ,ax = axes , bins = 30)
    plt.tight_layout()

<h2 style="background-color:powderblue; color:red; font-family:verdana;text-align:center;border: 20px solid black;margin: 50px;font-weight: bold;">
   <br> Clustring Pipeline<br> 

In [ ]:
#prepare the values for clustring
X = df.values
#Scale the values
from sklearn.preprocessing import MinMaxScaler
Sc = MinMaxScaler()
X = Sc.fit_transform(X)
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)
#Clustring usign Kmeans 
from sklearn.cluster import KMeans
clustering = KMeans(n_clusters=4).fit(principalComponents)
labels  = clustering.labels_

In [ ]:
fig , ax  = plt.subplots(figsize = (15,6))
custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)
sns.scatterplot(x = principalComponents[: , 0] , y = principalComponents[: , 1] ,
                hue  = labels , style = labels ,  palette="Set1" 
                , ax = ax).set_title("Clustring Groups" , fontsize = 18)

ax.set(xlabel='principalComponents 01', ylabel='principalComponents 02');


<h2 style="background-color:powderblue; color:red; font-family:verdana;text-align:center;border: 20px solid black;margin: 50px;font-weight: bold;">
   <br> Visualizing :
    <br>Incume / Purchasing habbits/ Expenses <br> For Each Cluster<br>

In [ ]:
df['labels'] = labels
sns.set_theme(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})
    # Initialize the FacetGrid object
pal = sns.cubehelix_palette(10, rot=-.25, light=.7)

def label(x, color, label):
    ax = plt.gca()
    ax.text(0, .2, label, fontweight="bold", color=color,
            ha="left", va="center", transform=ax.transAxes)
columns = ['log_Income', 'Total_expenses','log_MntWines',
           'log_MntFruits', 'log_MntMeatProducts', 'log_MntFishProducts',
           'log_MntSweetProducts', 'log_MntGoldProds', 'log_NumDealsPurchases',
           'log_NumWebPurchases', 'log_NumCatalogPurchases',
           'log_NumStorePurchases']    
for column in columns:

    g = sns.FacetGrid(df, row="labels", hue="labels", aspect=20, height=.7, palette=pal);

    # Draw the densities in a few steps
    g.map(sns.kdeplot, column,
          bw_adjust=.5, clip_on=False,
          fill=True, alpha=1, linewidth=1.5)
    g.map(sns.kdeplot, column, clip_on=False, color="w", lw=2, bw_adjust=.5)
    g.map(plt.axhline, y=0, lw=2, clip_on=False)


    g.map(label, column)

    # Set the subplots to overlap
    g.fig.subplots_adjust(hspace=-.25)

    # Remove axes details that don't play well with overlap
    g.set_titles("")
    g.set(yticks=[])
    g.despine(bottom=True, left=True)

<h2 style="background-color:powderblue; color:red; font-family:verdana;text-align:center;border: 20px solid black;margin: 50px;font-weight: bold;">
   <br> Visualizing :
    <br>Enrollment duration<br> For Each Cluster<br>

In [ ]:
columns = ['new_enrollement_less_year',
       'Range_one_year_enrollment', 'Two_years_enrollement',
       'Long_term_enrollement']
for column in columns:
    fig , ax = plt.subplots()
    sns.countplot(x= column , data  = df , hue  = 'labels' , ax = ax)


<h2 style="background-color:powderblue; color:red; font-family:verdana;text-align:center;border: 20px solid black;margin: 50px;font-weight: bold;">
   <br> Conclusion<br>

<ul style="background-color:powderblue; color:black; font-family:verdana;border: 20px solid black;margin: 10px;"><br><br>
    <li> <strong>Cluster 0 and Cluster 3:</strong> Low income , Low expenses (un-active customers)</li>
    <li> <strong>Cluster 1:</strong>  High income , High expenses , most of them newely enrolled ( Promising cutomers) </li>
    <li> <strong>Cluster 2:</strong> High income , High expenses ,old customers (loyal customers)</li>
<br><br>
    </ul>